In [55]:
import pandas as pd
import numpy as np

In [56]:
from sklearn.model_selection import train_test_split

housing_data = pd.read_csv('./melb_data.csv')

target_variable = housing_data['Price']
housing_data.drop(columns=['Price'], inplace=True)



x_train, x_validation, y_train, y_validation = train_test_split(housing_data, target_variable, train_size=0.8, test_size=0.2, random_state=0)

x_train.head()

,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
12167,St Kilda,11/22 Charnwood Cr,1,u,S,hockingstuart,29/07/2017,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,Port Phillip,-37.85984,144.9867,Southern Metropolitan,13240.0
6524,Williamstown,18 James St,2,h,SA,Hunter,17/09/2016,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,Hobsons Bay,-37.85800,144.9005,Western Metropolitan,6380.0
8413,Sunshine,10 Dundalk St,3,h,S,Barry,8/04/2017,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,Brimbank,-37.79880,144.8220,Western Metropolitan,3755.0
2919,Glenroy,1/2 Prospect St,3,u,SP,Brad,18/06/2016,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,Moreland,-37.70830,144.9158,Northern Metropolitan,8870.0
6043,Sunshine North,35 Furlong Rd,3,h,S,First,22/05/2016,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,Brimbank,-37.76230,144.8272,Western Metropolitan,4217.0


In [57]:
categorical_columns = [col for col in x_train.columns if x_train[col].dtype == 'object' and x_train[col].nunique() < 10]

categorical_columns

['Type', 'Method', 'Regionname']

In [58]:
numerical_columns = [col for col in x_train.columns if x_train[col].dtype in ['int64', 'float64']]

numerical_columns

['Rooms',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Car',
 'Landsize',
 'BuildingArea',
 'YearBuilt',
 'Lattitude',
 'Longtitude',
 'Propertycount']

In [59]:
full_cols = categorical_columns + numerical_columns

x_train = x_train[full_cols]
x_validation = x_validation[full_cols]

In [60]:
x_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


In [61]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# defines the method we are using to handle imputation of numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# defines the pipeline for handling categorical data,  both imputation and encoding
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Bundles the preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_columns),  ('cat', categorical_transformer, categorical_columns)])

In [62]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [63]:
from sklearn.metrics import mean_absolute_error

my_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

In [64]:
my_pipeline.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['Rooms', 'Distance',
                                                   'Postcode', 'Bedroom2',
                                                   'Bathroom', 'Car',
                                                   'Landsize', 'BuildingArea',
                                                   'YearBuilt', 'Lattitude',
                                                   'Longtitude',
                                                   'Propertycount']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('oneh

In [65]:
predictions = my_pipeline.predict(x_validation)

predictions

array([1673192.76      ,  895763.33333333,  572680.        , ...,
       1285440.        , 1259235.        , 1077630.        ])

In [66]:
mean_absolute_error(predictions, y_validation)

160679.18917034855

In [67]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

# transformer for numerical columns
numerical_transformer = SimpleImputer()

# transformer for categorical columns
categorical_transformer = Pipeline(steps=[('imputation', SimpleImputer(strategy='most_frequent')), ('ordinal', OrdinalEncoder())])

preprocessor = ColumnTransformer(transformers=[('numerical_variable_columns', numerical_transformer, numerical_columns), ('categorical_variable_columns', categorical_transformer, categorical_columns)])

model_ = RandomForestRegressor(n_estimators=155, random_state=0)

pipeline = Pipeline(steps=[('preprocessing', preprocessor), ('predictor', model_)])

pipeline.fit(x_train, y_train)

predictions = pipeline.predict(x_validation)

mean_absolute_error(predictions, y_validation)

161658.25385381954